In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input
import numpy as np
import pandas as pd

batch_size = 64
epochs = 100
latent_dim = 128
num_samples = 127606
filename = 'English_hindi.csv'

In [25]:
# vectorize the data
input_texts = []
target_texts = []
input_characters = []
target_characters = []

In [26]:
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)   
    except:        
        return ''
    
df = pd.read_csv(filename , converters={'English': convert_dtype,'Hindi': convert_dtype})
input_texts = df['English'].values.tolist()
target_texts = df['Hindi'].values.tolist()

In [27]:
for i in range(len(target_texts)):
    target_texts[i] = '\t' + target_texts[i] + '\n'
    for char in input_texts[i]:
        if char not in input_characters:
            input_characters.append(char)
    for char in target_texts[i]:
        if char not in target_characters:
            target_characters.append(char)

In [28]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [29]:
print('i/p characters : {}'.format(len(input_characters)))
print('targert characters : {}'.format(len(target_characters)))
print('encoder characters : {}'.format(num_encoder_tokens))
print('decoder characters : {}'.format(num_decoder_tokens))
print('max encoder : {}'.format(max_encoder_seq_len))
print('max decoder : {}'.format(max_decoder_seq_len))

i/p characters : 180
targert characters : 245
encoder characters : 180
decoder characters : 245
max encoder : 2239
max decoder : 2046


In [30]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '>': 30, '?': 31, '@': 32, 'A': 33, 'B': 34, 'C': 35, 'D': 36, 'E': 37, 'F': 38, 'G': 39, 'H': 40, 'I': 41, 'J': 42, 'K': 43, 'L': 44, 'M': 45, 'N': 46, 'O': 47, 'P': 48, 'Q': 49, 'R': 50, 'S': 51, 'T': 52, 'U': 53, 'V': 54, 'W': 55, 'X': 56, 'Y': 57, 'Z': 58, '[': 59, '\\': 60, ']': 61, '^': 62, '_': 63, '`': 64, 'a': 65, 'b': 66, 'c': 67, 'd': 68, 'e': 69, 'f': 70, 'g': 71, 'h': 72, 'i': 73, 'j': 74, 'k': 75, 'l': 76, 'm': 77, 'n': 78, 'o': 79, 'p': 80, 'q': 81, 'r': 82, 's': 83, 't': 84, 'u': 85, 'v': 86, 'w': 87, 'x': 88, 'y': 89, 'z': 90, '{': 91, '|': 92, '}': 93, '~': 94, '£': 95, '°': 96, '±': 97, '²': 98, 'µ': 99, '½': 100, 'Á': 101, 'É': 102, 'Í': 103, 'á': 104, 'é': 105, 'í': 106, 'ó': 107, 'ā': 108, 'ē': 109, 'ī': 110

In [31]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')

MemoryError: Unable to allocate 192. GiB for an array with shape (127607, 2239, 180) and data type float32

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i,t-1,target_token_index[char]] = 1.
    decoder_input_data[i,t+1:,target_token_index[' ']] = 1.
    decoder_target_data[i,t:,target_token_index[' ']] = 1.

In [ ]:
encoder_input_data[0].shape

In [ ]:
# Define an input sequence and process it
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c=encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states
encoder_states=[state_h,state_c]

In [ ]:
# Set up decoder , using ` encoder_states` as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well . we don't use the 
# return states in the training model , but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs= decoder_dense(decoder_outputs)

In [ ]:
# Define the model that will turn 
# `encoder_input_data` and `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics= ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size= batch_size, epochs = epochs , 
           validation_split=0.2)


In [ ]:
#model.save('trained_model.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [ ]:
for seq_index in range(20):
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print(input_texts[seq_index])
    print(decoded_sentence)